# Pipeline de Preprocesamiento de Datos de Rasgos Cuantitativos de Arroz

**Objetivo:** Realizar un análisis exploratorio de datos (EDA) robusto y preparar los datos de rasgos fenotípicos de arroz para análisis posteriores.

**Autor:** [Tu nombre]  
**Fecha:** Octubre 2025  
**Dataset:** Rasgos cuantitativos de arroz (2,266 accesiones × 12 rasgos)

---

## Índice
1. [Configuración e Importación de Librerías](#1-configuración)
2. [Carga de Datos](#2-carga-de-datos)
3. [Exploración Inicial](#3-exploración-inicial)
4. [Limpieza de Datos](#4-limpieza)
5. [Tratamiento de Valores Faltantes](#5-valores-faltantes)
6. [Detección de Outliers](#6-outliers)
7. [Estandarización](#7-estandarización)
8. [Enriquecimiento con Metadatos](#8-metadatos)
9. [Alineación con Datos Genómicos](#9-alineación)
10. [Exportación de Resultados](#10-exportación)

## 1. Configuración e Importación de Librerías

Configuración del entorno de trabajo e importación de las librerías necesarias para el análisis.

In [ ]:
# ============================================================================
# IMPORTACIÓN DE LIBRERÍAS
# ============================================================================

# Manipulación de datos
import pandas as pd
import numpy as np
from pathlib import Path

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Geolocalización
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pycountry

# Utilidades
import json
import time
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN DE VISUALIZACIÓN
# ============================================================================

# Configuración de pandas
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)
pd.set_option("display.precision", 2)

# Configuración de seaborn
sns.set_style("whitegrid")
sns.set_context("notebook")

# Configuración de matplotlib
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "figure.dpi": 100,
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "figure.titlesize": 16
})

print("✓ Librerías importadas correctamente")
print(f"✓ Pandas versión: {pd.__version__}")
print(f"✓ NumPy versión: {np.__version__}")

## 2. Carga de Datos

Carga del dataset de rasgos cuantitativos y configuración de rutas de trabajo.

In [ ]:
# ============================================================================
# DEFINICIÓN DE RUTAS
# ============================================================================

ROOT = Path("..")
DATA_DIR = ROOT / "data"
TRAITS_PATH = DATA_DIR / "trait_data" / "quantitative_traits.csv"
REPORTS_DIR = ROOT / "reports"

# Crear directorios si no existen
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================================
# CARGA DE DATOS
# ============================================================================

df = pd.read_csv(TRAITS_PATH, index_col=0)

# Limpieza inicial de nombres de columnas y tipos de datos
df.columns = df.columns.str.strip()
df = df.apply(pd.to_numeric, errors="coerce")
df.index = df.index.astype(str).str.strip()

print(f"✓ Dataset cargado exitosamente")
print(f"  Dimensiones: {df.shape[0]:,} muestras × {df.shape[1]} rasgos")
print(f"  Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 3. Exploración Inicial

### 3.1 Descripción de Rasgos

El dataset contiene 12 rasgos cuantitativos de arroz:

| Código | Descripción en Inglés | Descripción en Español | Unidad |
|--------|----------------------|------------------------|--------|
| **CUDI_REPRO** | Culm diameter | Diámetro del tallo | mm |
| **CULT_REPRO** | Culm length | Longitud del tallo | cm |
| **CUNO_REPRO** | Culm number | Número de tallos | unidades |
| **GRLT** | Grain length | Longitud del grano | mm |
| **GRWD** | Grain width | Ancho del grano | mm |
| **GRWT100** | 100-grain weight | Peso de 100 granos | g |
| **HDG_80HEAD** | Heading date (80% flowering) | Fecha de floración (80%) | días |
| **LIGLT** | Ligule length | Longitud de la lígula | mm |
| **LLT** | Leaf length | Longitud de la hoja | cm |
| **LWD** | Leaf width | Ancho de la hoja | cm |
| **PLT_POST** | Panicle length | Longitud de la panícula | cm |
| **SDHT** | Seedling height | Altura de plántula | cm |

In [ ]:
# ============================================================================
# RESUMEN ESTADÍSTICO
# ============================================================================

print("=" * 80)
print("RESUMEN DEL DATASET")
print("=" * 80)

print(f"\n📊 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas\n")

print("Columnas del dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n" + "=" * 80)
print("PRIMERAS 5 MUESTRAS")
print("=" * 80)
display(df.head())

print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 80)
display(df.describe().T)

### 3.2 Análisis de Valores Faltantes

In [ ]:
# ============================================================================
# ANÁLISIS DE VALORES FALTANTES
# ============================================================================

# Calcular porcentaje de valores faltantes por rasgo
missing_traits = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)

print("=" * 80)
print("ANÁLISIS DE VALORES FALTANTES POR RASGO")
print("=" * 80)

print(f"\n{'Rasgo':<15} {'Faltantes':<12} {'Porcentaje'}")
print("-" * 45)
for trait, pct in missing_traits.items():
    print(f"{trait:<15} {df[trait].isna().sum():<12} {pct:>6.2f}%")

# Calcular porcentaje de valores faltantes por muestra
missing_samples = df.isna().sum(axis=1)

print("\n" + "=" * 80)
print("DISTRIBUCIÓN DE VALORES FALTANTES POR MUESTRA")
print("=" * 80)
print(f"\nMuestras sin valores faltantes: {(missing_samples == 0).sum():,} ({(missing_samples == 0).sum()/len(df)*100:.1f}%)")
print(f"Muestras con 1-3 faltantes: {((missing_samples > 0) & (missing_samples <= 3)).sum():,}")
print(f"Muestras con 4-6 faltantes: {((missing_samples > 3) & (missing_samples <= 6)).sum():,}")
print(f"Muestras con >6 faltantes: {(missing_samples > 6).sum():,}")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras por rasgo
missing_traits.plot(kind='barh', ax=axes[0], color='coral')
axes[0].set_xlabel('Porcentaje de valores faltantes (%)')
axes[0].set_title('Valores Faltantes por Rasgo')
axes[0].grid(axis='x', alpha=0.3)

# Histograma por muestra
axes[1].hist(missing_samples, bins=13, color='skyblue', edgecolor='black')
axes[1].set_xlabel('Número de rasgos faltantes')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Faltantes por Muestra')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(REPORTS_DIR / 'missing_values_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Gráfico guardado en: {REPORTS_DIR / 'missing_values_analysis.png'}")

## 4. Limpieza de Datos

### 4.1 Eliminación de Duplicados y Columnas Constantes

In [ ]:
# ============================================================================
# LIMPIEZA: DUPLICADOS Y COLUMNAS CONSTANTES
# ============================================================================

print("=" * 80)
print("LIMPIEZA DE DATOS")
print("=" * 80)

# Verificar duplicados
duplicates = df.duplicated()
print(f"\n📌 Filas duplicadas encontradas: {duplicates.sum()}")

if duplicates.sum() > 0:
    print("   Eliminando duplicados...")
    df = df[~duplicates]

# Verificar columnas constantes
constant_cols = [col for col in df.columns if df[col].nunique(dropna=True) <= 1]
print(f"\n📌 Columnas constantes encontradas: {len(constant_cols)}")

if constant_cols:
    print(f"   Columnas: {constant_cols}")
    print("   Eliminando columnas constantes...")
    df = df.drop(columns=constant_cols)

print(f"\n✓ Dimensiones después de limpieza: {df.shape[0]:,} × {df.shape[1]}")

## 5. Tratamiento de Valores Faltantes

### 5.1 Filtrado por Umbrales

Aplicamos umbrales de calidad para eliminar rasgos y muestras con exceso de valores faltantes:
- **Rasgos:** Eliminar si >20% de valores faltantes
- **Muestras:** Eliminar si >50% de valores faltantes

In [ ]:
# ============================================================================
# FILTRADO POR UMBRALES DE VALORES FALTANTES
# ============================================================================

print("=" * 80)
print("FILTRADO POR UMBRALES")
print("=" * 80)

# Umbrales
TRAIT_THRESHOLD = 0.20  # 20% máximo de faltantes por rasgo
SAMPLE_THRESHOLD = 0.50  # 50% máximo de faltantes por muestra

original_shape = df.shape

# 1. Filtrar rasgos
trait_missing_pct = df.isna().mean()
traits_to_keep = trait_missing_pct[trait_missing_pct <= TRAIT_THRESHOLD].index
traits_removed = df.columns.difference(traits_to_keep)

print(f"\n1️⃣ FILTRADO DE RASGOS (umbral: {TRAIT_THRESHOLD*100:.0f}%)")
print(f"   Rasgos originales: {df.shape[1]}")
print(f"   Rasgos eliminados: {len(traits_removed)}")
if len(traits_removed) > 0:
    for trait in traits_removed:
        pct = trait_missing_pct[trait] * 100
        print(f"     - {trait}: {pct:.1f}% faltantes")

df = df[traits_to_keep]

# 2. Filtrar muestras
sample_missing_pct = df.isna().mean(axis=1)
samples_to_keep = sample_missing_pct[sample_missing_pct <= SAMPLE_THRESHOLD].index
samples_removed = len(df) - len(samples_to_keep)

print(f"\n2️⃣ FILTRADO DE MUESTRAS (umbral: {SAMPLE_THRESHOLD*100:.0f}%)")
print(f"   Muestras originales: {original_shape[0]:,}")
print(f"   Muestras eliminadas: {samples_removed:,}")

df = df.loc[samples_to_keep]

print(f"\n✓ Dimensiones después del filtrado: {df.shape[0]:,} × {df.shape[1]}")
print(f"  Reducción: {(1 - df.shape[0]/original_shape[0])*100:.1f}% en muestras")

### 5.2 Imputación de Valores Faltantes

Imputamos los valores faltantes restantes usando la **mediana** de cada rasgo (método robusto ante outliers).

In [ ]:
# ============================================================================
# IMPUTACIÓN POR MEDIANA
# ============================================================================

print("=" * 80)
print("IMPUTACIÓN DE VALORES FALTANTES")
print("=" * 80)

# Contar faltantes antes de imputación
missing_before = df.isna().sum().sum()
print(f"\nValores faltantes antes de imputación: {missing_before:,}")

# Imputación por mediana
df_imputed = df.fillna(df.median())

# Verificar imputación
missing_after = df_imputed.isna().sum().sum()
print(f"Valores faltantes después de imputación: {missing_after:,}")

if missing_after == 0:
    print("\n✓ Todos los valores faltantes han sido imputados exitosamente")
    df = df_imputed
else:
    print(f"\n⚠ Aún quedan {missing_after} valores faltantes (posibles columnas vacías)")

# Mostrar medianas utilizadas
print("\nMedianas utilizadas para imputación:")
print(df.median().to_string())

## 6. Detección de Outliers

Identificación de valores atípicos usando el método del **rango intercuartílico (IQR)**.

In [ ]:
# ============================================================================
# DETECCIÓN DE OUTLIERS (MÉTODO IQR)
# ============================================================================

print("=" * 80)
print("DETECCIÓN DE OUTLIERS")
print("=" * 80)

def detect_outliers_iqr(series, multiplier=1.5):
    """
    Detecta outliers usando el método IQR.
    
    Parámetros:
    -----------
    series : pd.Series
        Serie de datos numéricos
    multiplier : float
        Multiplicador del IQR (1.5 para outliers moderados, 3 para extremos)
    
    Retorna:
    --------
    pd.Series : Máscara booleana de outliers
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return (series < lower_bound) | (series > upper_bound)

# Detectar outliers por rasgo
outlier_summary = {}
for col in df.columns:
    outliers = detect_outliers_iqr(df[col])
    outlier_summary[col] = {
        'count': outliers.sum(),
        'percentage': outliers.sum() / len(df) * 100
    }

# Mostrar resumen
print(f"\n{'Rasgo':<15} {'Outliers':<10} {'Porcentaje'}")
print("-" * 40)
for trait, stats in outlier_summary.items():
    print(f"{trait:<15} {stats['count']:<10} {stats['percentage']:>6.2f}%")

total_outliers = sum(s['count'] for s in outlier_summary.values())
print(f"\nTotal de outliers detectados: {total_outliers:,}")
print(f"Porcentaje del dataset: {total_outliers/(df.shape[0]*df.shape[1])*100:.2f}%")

# Visualización
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for i, col in enumerate(df.columns):
    axes[i].boxplot(df[col], vert=True)
    axes[i].set_title(col, fontsize=10, fontweight='bold')
    axes[i].set_ylabel('Valor')
    axes[i].grid(axis='y', alpha=0.3)
    
    # Añadir conteo de outliers
    n_outliers = outlier_summary[col]['count']
    axes[i].text(0.5, 0.95, f'Outliers: {n_outliers}',
                transform=axes[i].transAxes,
                ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                fontsize=8)

plt.suptitle('Distribución de Rasgos y Detección de Outliers', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(REPORTS_DIR / 'outliers_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Gráfico guardado en: {REPORTS_DIR / 'outliers_boxplots.png'}")
print("\n⚠ Nota: Los outliers se mantienen en el dataset (pueden ser variación biológica real)")

## 7. Estandarización de Datos

Estandarización Z-score (media=0, desviación estándar=1) para análisis posteriores.

In [ ]:
# ============================================================================
# ESTANDARIZACIÓN (Z-SCORE)
# ============================================================================

print("=" * 80)
print("ESTANDARIZACIÓN DE DATOS")
print("=" * 80)

# Guardar copia de datos sin estandarizar
df_raw = df.copy()

# Estandarización
df_scaled = (df - df.mean()) / df.std()

# Verificar estandarización
print("\nVerificación de estandarización:")
print(f"\n{'Rasgo':<15} {'Media':<10} {'Desv. Std'}")
print("-" * 40)
for col in df_scaled.columns:
    print(f"{col:<15} {df_scaled[col].mean():>9.6f} {df_scaled[col].std():>10.6f}")

print("\n✓ Datos estandarizados correctamente (media ≈ 0, std ≈ 1)")

# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Antes de estandarización
df_raw.boxplot(ax=axes[0], rot=45)
axes[0].set_title('Datos Originales', fontweight='bold')
axes[0].set_ylabel('Valor')
axes[0].grid(axis='y', alpha=0.3)

# Después de estandarización
df_scaled.boxplot(ax=axes[1], rot=45)
axes[1].set_title('Datos Estandarizados (Z-score)', fontweight='bold')
axes[1].set_ylabel('Z-score')
axes[1].grid(axis='y', alpha=0.3)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=0.8, label='Media=0')
axes[1].legend()

plt.tight_layout()
plt.savefig(REPORTS_DIR / 'standardization_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Gráfico guardado en: {REPORTS_DIR / 'standardization_comparison.png'}")

## 8. Enriquecimiento con Metadatos

### 8.1 Carga de Metadatos de Pasaporte

In [ ]:
# ============================================================================
# CARGA DE METADATOS DE PASAPORTE
# ============================================================================

print("=" * 80)
print("ENRIQUECIMIENTO CON METADATOS")
print("=" * 80)

# Cargar datos de pasaporte
PASSPORT_PATH = DATA_DIR / "passport_data" / "passport.csv"

try:
    passport = pd.read_csv(PASSPORT_PATH, index_col=0)
    passport.index = passport.index.astype(str).str.strip()
    
    print(f"\n✓ Metadatos cargados: {passport.shape[0]:,} accesiones")
    print(f"  Columnas disponibles: {', '.join(passport.columns[:5])}...")
    
    # Fusionar con datos de rasgos
    df_enriched = df.join(passport, how='left')
    
    print(f"\n✓ Datos enriquecidos: {df_enriched.shape[0]:,} × {df_enriched.shape[1]} columnas")
    
    # Resumen de información geográfica
    if 'country' in df_enriched.columns:
        print(f"\nDistribución geográfica:")
        country_counts = df_enriched['country'].value_counts().head(10)
        for country, count in country_counts.items():
            print(f"  {country}: {count:,} accesiones")
    
    df_clean = df_enriched.copy()
    
except FileNotFoundError:
    print(f"\n⚠ Archivo de metadatos no encontrado: {PASSPORT_PATH}")
    print("  Continuando sin enriquecimiento...")
    df_clean = df.copy()

### 8.2 Geocodificación de Países

**Nota:** Esta sección requiere conexión a internet y puede tardar varios minutos.

In [ ]:
# ============================================================================
# GEOCODIFICACIÓN (OPCIONAL)
# ============================================================================

print("=" * 80)
print("GEOCODIFICACIÓN DE PAÍSES")
print("=" * 80)

ENABLE_GEOCODING = False  # Cambiar a True para ejecutar

if ENABLE_GEOCODING and 'country' in df_clean.columns:
    
    def normalize_country(name):
        """Normaliza nombres de países usando pycountry"""
        try:
            return pycountry.countries.lookup(name).name
        except LookupError:
            return name
    
    def get_coordinates(country_name, geolocator):
        """Obtiene coordenadas del centroide de un país"""
        try:
            location = geolocator.geocode(country_name, timeout=10)
            if location is None:
                location = geolocator.geocode(country_name.replace('_', ' '), timeout=10)
            
            if location:
                return location.latitude, location.longitude
            return None, None
        except Exception as e:
            print(f"  Error: {country_name} - {str(e)}")
            return None, None
    
    # Inicializar geocodificador
    geolocator = Nominatim(user_agent="rice_analysis_2025")
    
    # Obtener países únicos
    countries = df_clean['country'].dropna().unique()
    coordinates_map = {}
    
    print(f"\nObteniendo coordenadas para {len(countries)} países...\n")
    
    for i, country in enumerate(countries, 1):
        lat, lon = get_coordinates(country, geolocator)
        if lat and lon:
            coordinates_map[country] = [lat, lon]
            print(f"  {i:2d}. ✓ {country:<30} {lat:>7.2f}°, {lon:>7.2f}°")
        else:
            print(f"  {i:2d}. ✗ {country:<30} (no encontrado)")
        time.sleep(1)  # Respetar límites de API
    
    # Guardar coordenadas
    coords_path = DATA_DIR / "country_coordinates.json"
    with open(coords_path, 'w', encoding='utf-8') as f:
        json.dump(coordinates_map, f, indent=4, ensure_ascii=False)
    
    print(f"\n✓ Coordenadas guardadas en: {coords_path}")
    
    # Añadir coordenadas al DataFrame
    df_clean['latitude'] = df_clean['country'].map(
        lambda x: coordinates_map[x][0] if x in coordinates_map else None
    )
    df_clean['longitude'] = df_clean['country'].map(
        lambda x: coordinates_map[x][1] if x in coordinates_map else None
    )
    
    print(f"\n✓ Accesiones geocodificadas: {df_clean['latitude'].notna().sum():,} de {len(df_clean):,}")

else:
    print("\n⊘ Geocodificación deshabilitada")
    print("  Para habilitar: establece ENABLE_GEOCODING = True")

## 9. Alineación con Datos Genómicos

Alineación de muestras entre datos fenotípicos y genómicos para análisis integrados.

In [ ]:
# ============================================================================
# ALINEACIÓN CON DATOS GENÓMICOS
# ============================================================================

print("=" * 80)
print("ALINEACIÓN CON DATOS GENÓMICOS")
print("=" * 80)

GENO_PATH = DATA_DIR / "genotype_data" / "genotype.csv"

try:
    # Cargar datos genómicos
    geno = pd.read_csv(GENO_PATH, index_col=0)
    geno.index = geno.index.astype(str).str.strip()
    
    print(f"\n✓ Datos genómicos cargados: {geno.shape[0]:,} × {geno.shape[1]:,} SNPs")
    
    # Identificar muestras comunes
    traits_ids = set(df_clean.index)
    geno_ids = set(geno.index)
    common_ids = traits_ids.intersection(geno_ids)
    
    print(f"\nAnálisis de superposición:")
    print(f"  Muestras en rasgos: {len(traits_ids):,}")
    print(f"  Muestras en genotipos: {len(geno_ids):,}")
    print(f"  Muestras en común: {len(common_ids):,}")
    print(f"  Solo en rasgos: {len(traits_ids - geno_ids):,}")
    print(f"  Solo en genotipos: {len(geno_ids - traits_ids):,}")
    
    if len(common_ids) == 0:
        print("\n⚠ ERROR: No hay muestras en común")
        print("  Verificar formato de IDs de muestra")
    else:
        # Crear subconjuntos alineados
        common_ids_sorted = sorted(common_ids)
        traits_aligned = df_clean.loc[common_ids_sorted].copy()
        geno_aligned = geno.loc[common_ids_sorted].copy()
        
        print(f"\n✓ Alineación exitosa:")
        print(f"  Rasgos alineados: {traits_aligned.shape}")
        print(f"  Genotipos alineados: {geno_aligned.shape}")
        
        # Guardar datasets alineados
        traits_aligned.to_csv(REPORTS_DIR / "traits_aligned.csv")
        geno_aligned.to_csv(REPORTS_DIR / "genotypes_aligned.csv")
        
        print(f"\n✓ Datasets alineados guardados en {REPORTS_DIR}")

except FileNotFoundError:
    print(f"\n⚠ Archivo de genotipos no encontrado: {GENO_PATH}")
    print("  Omitiendo alineación genómica...")
except Exception as e:
    print(f"\n⚠ Error durante alineación: {str(e)}")

## 10. Exportación de Resultados

Guardado de todos los datasets procesados y reporte de resumen.

In [ ]:
# ============================================================================
# EXPORTACIÓN DE RESULTADOS
# ============================================================================

print("=" * 80)
print("EXPORTACIÓN DE RESULTADOS")
print("=" * 80)

# Crear diccionario de datasets
datasets = {
    'traits_raw': df_raw,
    'traits_clean': df_clean,
    'traits_scaled': df_scaled
}

# Guardar datasets
print("\nGuardando datasets procesados...")
for name, dataset in datasets.items():
    output_path = REPORTS_DIR / f"{name}.csv"
    dataset.to_csv(output_path)
    print(f"  ✓ {name}.csv - {dataset.shape[0]:,} × {dataset.shape[1]} columnas")

# Crear reporte de resumen
summary_report = {
    'Información General': {
        'Fecha de procesamiento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'Dataset original': str(TRAITS_PATH),
        'Dimensiones originales': f"{original_shape[0]:,} × {original_shape[1]}",
        'Dimensiones finales': f"{df_clean.shape[0]:,} × {df_clean.shape[1]}"
    },
    'Limpieza': {
        'Duplicados eliminados': duplicates.sum(),
        'Columnas constantes eliminadas': len(constant_cols) if constant_cols else 0,
        'Muestras filtradas por umbral': samples_removed,
        'Rasgos filtrados por umbral': len(traits_removed) if len(traits_removed) > 0 else 0
    },
    'Valores Faltantes': {
        'Total antes de imputación': missing_before,
        'Total después de imputación': missing_after,
        'Método de imputación': 'Mediana'
    },
    'Calidad de Datos': {
        'Outliers detectados': total_outliers,
        'Porcentaje de outliers': f"{total_outliers/(df.shape[0]*df.shape[1])*100:.2f}%",
        'Datos estandarizados': 'Sí (Z-score)'
    }
}

# Guardar reporte en JSON
report_path = REPORTS_DIR / "preprocessing_summary.json"
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(summary_report, f, indent=4, ensure_ascii=False)

print(f"\n✓ Reporte de resumen guardado: {report_path}")

# Mostrar resumen en pantalla
print("\n" + "=" * 80)
print("RESUMEN DEL PREPROCESAMIENTO")
print("=" * 80)

for section, items in summary_report.items():
    print(f"\n{section}:")
    for key, value in items.items():
        print(f"  • {key}: {value}")

print("\n" + "=" * 80)
print("✓ PREPROCESAMIENTO COMPLETADO EXITOSAMENTE")
print("=" * 80)
print(f"\nTodos los archivos guardados en: {REPORTS_DIR.absolute()}")

---

## Conclusiones del Preprocesamiento

### Resultados Clave:

1. **Calidad de datos mejorada:** Se eliminaron duplicados, valores constantes y muestras/rasgos con exceso de faltantes
2. **Imputación robusta:** Uso de mediana para resistencia ante outliers
3. **Estandarización:** Datos preparados para análisis multivariado
4. **Enriquecimiento:** Metadatos de pasaporte integrados
5. **Alineación genómica:** Muestras sincronizadas con datos moleculares

### Archivos Generados:

- `traits_raw.csv` - Datos originales limpios
- `traits_clean.csv` - Datos imputados con metadatos
- `traits_scaled.csv` - Datos estandarizados (Z-score)
- `traits_aligned.csv` - Rasgos alineados con genotipos
- `genotypes_aligned.csv` - Genotipos alineados
- `preprocessing_summary.json` - Reporte detallado
- Gráficos: `missing_values_analysis.png`, `outliers_boxplots.png`, `standardization_comparison.png`

### Próximos Pasos:

1. Análisis de componentes principales (PCA)
2. Análisis de correlaciones entre rasgos
3. Estudios de asociación genómica (GWAS)
4. Modelado predictivo de rasgos

---

**Fecha:** Octubre 2025  
**Notebook preparado para presentación profesional** ✨